In [ ]:
# Cell 1 — SparkSession & JDBC Setup
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, avg
import os

DB_CONFIG = {
    "host":     "localhost",
    "port":     "5432",
    "user":     "postgres",
    "password": "1234",
    "db":       "tourism",
    "driver":   "org.postgresql.Driver"
}

spark = (
    SparkSession.builder
    .appName("03_agg_occupancy")
    .config("spark.jars.packages","org.postgresql:postgresql:42.6.0")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

# Cell 2 — Read three staging occupancy tables
jdbc_url = f"jdbc:postgresql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['db']}"
props = {"user": DB_CONFIG["user"], "password": DB_CONFIG["password"], "driver": DB_CONFIG["driver"]}

arm = spark.read.jdbc(jdbc_url, "staging_occ_estat_tour_occ_arm_en", properties=props)\
            .withColumn("TIME_PERIOD", date_format("TIME_PERIOD","yyyy-MM"))\
            .select("geo","TIME_PERIOD","log_OBS_VALUE")\
            .withColumnRenamed("log_OBS_VALUE","log_arrivals")

nim = spark.read.jdbc(jdbc_url, "staging_occ_estat_tour_occ_nim_en", properties=props)\
            .withColumn("TIME_PERIOD", date_format("TIME_PERIOD","yyyy-MM"))\
            .select("geo","TIME_PERIOD","log_OBS_VALUE")\
            .withColumnRenamed("log_OBS_VALUE","log_nights_spent")

mnor = spark.read.jdbc(jdbc_url, "staging_occ_estat_tour_occ_mnor_en", properties=props)\
            .withColumn("TIME_PERIOD", date_format("TIME_PERIOD","yyyy-MM"))\
            .select("geo","TIME_PERIOD","OBS_VALUE")\
            .withColumnRenamed("OBS_VALUE","occupancy_rate")

# merge & yearly aggregate
merged = arm.join(nim, ["geo","TIME_PERIOD"]).join(mnor, ["geo","TIME_PERIOD"])
yearly = (
    merged.withColumn("year", date_format("TIME_PERIOD","yyyy").cast("int"))
          .groupBy("geo","year")
          .agg(
             avg("occupancy_rate").alias("occupancy_rate"),
             avg("log_arrivals").alias("mean_log_arrivals"),
             avg("log_nights_spent").alias("mean_log_nights_spent")
          )
)

# write back to Postgres
yearly.write.mode("overwrite") \
       .jdbc(jdbc_url,"tourism_occupancy_yearly",properties=props)

print("tourism_occupancy_yearly written")
yearly.show(5,truncate=False)
